In [3]:
import re
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import spacy
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### **Data Reading**

In [11]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

In [12]:
train_data = train_data.dropna()

In [13]:
test_data = test_data.dropna()

### **Data Splitting**

In [14]:
x_train, y_train, x_test, y_test = train_data['text'], train_data['dialect'], test_data['text'], test_data['dialect']

### **Data Preparation**

In [60]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

x_train_sequence = tokenizer.texts_to_sequences(x_train)
x_test_sequence = tokenizer.texts_to_sequences(x_test)

In [17]:
max_sequence_len = 100
x_train_padded = pad_sequences(x_train_sequence, maxlen=max_sequence_len, padding='post')
x_test_padded = pad_sequences(x_test_sequence, maxlen=max_sequence_len, padding='post')

In [18]:
y_train_cat = np.eye(5)[y_train]
y_test_cat = np.eye(5)[y_test]

In [19]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

214200

### **Second Model**

In [20]:
model = Sequential([
    Embedding(vocab_size, 64),
    SimpleRNN(64),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          13708800  
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                8256      
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13721541 (52.34 MB)
Trainable params: 13721541 (52.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
history = model.fit(x_train_padded, y_train_cat, epochs=10, batch_size=32)

Epoch 1/10
3693/3693 [==============================] - 1391s 376ms/step - loss: 1.4558 - accuracy: 0.3899
Epoch 2/10
3693/3693 [==============================] - 1413s 383ms/step - loss: 1.4529 - accuracy: 0.3908
Epoch 3/10
3693/3693 [==============================] - 1400s 379ms/step - loss: 1.4527 - accuracy: 0.3909
Epoch 4/10
3693/3693 [==============================] - 1413s 382ms/step - loss: 1.4530 - accuracy: 0.3908
Epoch 5/10
3693/3693 [==============================] - 1435s 388ms/step - loss: 1.4528 - accuracy: 0.3909
Epoch 6/10
3693/3693 [==============================] - 1416s 384ms/step - loss: 1.4527 - accuracy: 0.3909
Epoch 7/10
3693/3693 [==============================] - 1403s 380ms/step - loss: 1.4528 - accuracy: 0.3909
Epoch 8/10
3693/3693 [==============================] - 1397s 378ms/step - loss: 1.4528 - accuracy: 0.3909
Epoch 9/10
3693/3693 [==============================] - 1410s 382ms/step - loss: 1.4527 - accuracy: 0.3909
Epoch 10/10
3693/3693 [==============

In [49]:
def dialect_predict(sentence):

  dialect = int(np.argmax(model.predict(sentence), axis=1)[0])

  if dialect == 0:
    country = "Egypt"

  elif  dialect == 1 :
    country = "Lebnanon"

  elif dialect ==   2 :
    country = "Libya"

  elif dialect == 3:
    country = "Morocco"

  else:
    country = "Sudan"

  return country

In [71]:
sentence = [" بالزاف "]
tokenizer.fit_on_texts(sentence)

x_train_sequence = tokenizer.texts_to_sequences(sentence)
x_train_sequence

[[28420]]

In [72]:
max_sequence_len = 100
x_train_padded = pad_sequences(x_train_sequence, maxlen=max_sequence_len, padding='post')
x_train_padded

array([[28420,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]], dtype=int32)

In [73]:
pred = model.predict(x_train_padded)

1/1 [==============================] - 0s 24ms/step


In [74]:
val = np.argmax(model.predict(x_train_padded), axis=1)[0]

1/1 [==============================] - 0s 32ms/step


In [75]:
dialect_predict(x_train_padded)

1/1 [==============================] - 0s 23ms/step


'Libya'